In [1]:
import os
def homePath(path):
    if path[0] == "~":
        return os.path.join(os.path.expanduser("~"), path.strip("~/"))
    else:
        return path

#Model Creation 1
#Figure out how to do versioning effectively
individualInput = False
combinedInputPath = "/home/jaredrussell/CPPMLGen"
#combinedInputPath = "/home/jaredrussell/CPPMLGenMini"
pathToTokenizedData = "C:\\Users\\mcall\\OneDrive\\Desktop\\DummyOutput\\Tokenizer\\"
pathToGradeData = "C:\\Users\\mcall\\OneDrive\\Desktop\\DummyOutput\\Grader\\"
GradesTokensName = ""


if not individualInput:
    pathToTokenizedData = os.path.join(combinedInputPath, "Tokens/")
    pathToGradeData = os.path.join(combinedInputPath, "Grades/")

if GradesTokensName == "":
    #Use newest folder for each

    #Get the newest folder for the tokens
    tokensFolders = os.listdir(pathToTokenizedData)
    tokensFolders.sort()
    pathToTokenizedData = os.path.join(pathToTokenizedData , tokensFolders[-1])

    #Get the newest folder for the grades
    gradesFolders = os.listdir(pathToGradeData)
    gradesFolders.sort()
    pathToGradeData = os.path.join(pathToGradeData,  gradesFolders[-1])
else:
    pathToTokenizedData = os.path.join(pathToTokenizedData, GradesTokensName)
    pathToGradeData = os.path.join(pathToGradeData, GradesTokensName)





modelOutputPath = "/home/jaredrussell/CPPMLGen/Models"



In [2]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import tokenizer_from_json
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, Dropout
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.models import model_from_json
from keras.models import Model
from matplotlib import pyplot as plt
import tensorflow as tf
import keras
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import tokenizer_from_json
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import keras.layers as layers
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.models import model_from_json
from keras.models import Model




from keras.utils import pad_sequences
import os
import sys

sys.path.append('./SCA-Tokenizer/')
import CLPTokenizer as CLPTokenizer



#Load the data
#tokenized data is in tokenizedData.pkl, has tokenizer obj in tokenizer.json

#Load the tokenizer
with open(pathToTokenizedData + "/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f, encoding='latin1')


#Load the tokenized data
with open(pathToTokenizedData + "/tokenizedData.pkl", "rb") as f:
    tokenizedData = pickle.load(f)

print (tokenizedData)

#Load the grade data
#gradeData is a dict with keys as the file names and values as the grades
with open(pathToGradeData + "/grades.pkl", "rb") as f:
    gradeData = pickle.load(f)

print (gradeData)

#load the group data
#with open(pathToTokenizedData + "/tokenGroupDataframe.pkl", "rb") as f:
#    tokenizedGroupData = pickle.load(f)

combinedDF = pd.merge(tokenizedData, gradeData, on = "Path")
#combinedDF = pd.merge(combinedDF, tokenizedGroupData, on = "Path")
print (combinedDF)







2023-11-29 21:07:18.558670: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 21:07:18.598696: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 21:07:18.598733: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 21:07:18.599977: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 21:07:18.605518: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 21:07:18.605890: I tensorflow/core/platform/cpu_feature_guard.cc:1

                                                tokenCode  \
0        9 9 9 9 276 2 485 1 9 276 2 485 1 9 9 133 394...   
1        9 9 9 9 276 2 485 1 9 276 2 485 1 9 9 133 394...   
2        9 9 9 9 276 2 485 1 9 276 2 485 1 9 9 133 394...   
3        9 9 9 9 276 2 485 1 9 276 2 485 1 9 9 133 394...   
4        9 9 9 9 276 2 485 1 9 276 2 485 1 9 9 133 394...   
...                                                   ...   
117086   45 20 550 65 31 46 35 28 28 9 119 706 690 65 ...   
117087   28 28 28 28 45 485 21 23 42 106 47 42 23 64 2...   
117088   28 28 28 28 45 485 21 23 42 106 47 42 23 64 2...   
117089   28 28 28 28 45 485 21 23 42 106 47 42 23 64 2...   
117093   45 15 15 15 15 15 15 15 81 15 15 15 133 283 3...   

                                                     Path  
0                                3385567/RBHook.cpp/4.cpp  
1                                3385567/RBHook.cpp/0.cpp  
2                                3385567/RBHook.cpp/5.cpp  
3                          

In [3]:
#Padding

maxLen = 500
minLen = 100
#get rid of the ones that are too long

#print (type(combinedDF["tokenCode"]))
#combinedDF = combinedDF[combinedDF["tokenCode"].apply(lambda x: len(x)) <= maxLen -1]

#shorten the ones that are too long FOR TESTING
combinedDF["tokenCode"] = combinedDF["tokenCode"].apply(lambda x: [int(i) for i in x.split()[:maxLen]])

#get rid of the ones that are too short DISABLED FOR TESTING
#combinedDF = combinedDF[combinedDF["tokenCode"].apply(lambda x: len(x)) > minLen]

#Pad the sequences
#combinedDF["tokenCode"] = combinedDF["tokenCode"].apply(lambda x: [int(i) for i in x.split()]).tolist()
combinedDF["tokenCode"] = pad_sequences(combinedDF["tokenCode"].tolist(), maxlen=maxLen, padding="post", truncating="post").tolist()
#combinedDF["tokenGroupCode"] = pad_sequences(combinedDF["tokenGroupCode"], maxlen = maxLen, padding = "post", truncating = "post").tolist()

print(combinedDF)

#only use 5% of the data
#combinedDF = combinedDF.sample(frac=0.001, random_state=1)

#48590 



                                               tokenCode  \
0      [9, 9, 9, 9, 276, 2, 485, 1, 9, 276, 2, 485, 1...   
1      [9, 9, 9, 9, 276, 2, 485, 1, 9, 276, 2, 485, 1...   
2      [9, 9, 9, 9, 276, 2, 485, 1, 9, 276, 2, 485, 1...   
3      [9, 9, 9, 9, 276, 2, 485, 1, 9, 276, 2, 485, 1...   
4      [9, 9, 9, 9, 276, 2, 485, 1, 9, 276, 2, 485, 1...   
...                                                  ...   
74335  [28, 9, 14, 9, 351, 147, 191, 50, 297, 9, 52, ...   
74336  [45, 20, 550, 65, 31, 46, 35, 28, 28, 9, 119, ...   
74337  [28, 28, 28, 28, 45, 485, 21, 23, 42, 106, 47,...   
74338  [28, 28, 28, 28, 45, 485, 21, 23, 42, 106, 47,...   
74339  [28, 28, 28, 28, 45, 485, 21, 23, 42, 106, 47,...   

                                                    Path  fileGrade  
0                               3385567/RBHook.cpp/4.cpp  68.333333  
1                               3385567/RBHook.cpp/0.cpp  55.000000  
2                               3385567/RBHook.cpp/5.cpp  71.666667  

In [4]:
#if (not gpu_detected):
#    print("GPU not detected, using CPU")

number_of_tokens = len(tokenizer.word_index) + 1
print (number_of_tokens)

model = Sequential()
model.add(Embedding(number_of_tokens, 64, input_length=maxLen))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Reshape((1, 64)))
model.add(layers.Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.38479930887149405))
model.add(layers.Bidirectional(LSTM(32)))
model.add(layers.Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_logarithmic_error', optimizer='Adam', metrics=['mse', 'mae', 'mape', 'accuracy'])





# convert inputs to numpy arrays
token_code = np.array(combinedDF["tokenCode"].tolist())
#token_group_code = np.array(combinedDF["tokenGroupCode"].tolist())
file_grade = np.array(combinedDF["fileGrade"].tolist())

hist = model.fit(token_code, file_grade, epochs=3, batch_size=32, verbose=1)


#Save the model in timestamp folder and with tokenizer
timestamp = str(pd.Timestamp.now()).replace(" ", "_").replace(":", "-").replace(".", "-")
if not os.path.exists(modelOutputPath):
    os.mkdir(modelOutputPath)
if not os.path.exists(modelOutputPath + "/" + timestamp):
    os.mkdir(modelOutputPath + "/" + timestamp)
model.save(modelOutputPath + "/" + timestamp + "/model.h5")
with open(modelOutputPath + "/" + timestamp + "/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
#make an archaive of the SCA-Tokenizer Folder
#get CWD
cwd = os.getcwd()
if os.path.exists(os.path.join(cwd, "SCA-Tokenizer")):
    os.system("tar -czvf \"" + modelOutputPath + "/" + timestamp + "/SCA-Tokenizer.tar.gz\" " + os.path.join(cwd, "SCA-Tokenizer"))

#Save the AutoGrader Folder
autoGraderDir = os.path.join(cwd, "../Auto-Grader/")
if os.path.exists(autoGraderDir):
    os.system("tar -czvf \"" + modelOutputPath + "/" + timestamp + "/Auto-Grader.tar.gz\" " + autoGraderDir)

#Save tokenizedGroupData
#with open(modelOutputPath + "/" + timestamp + "/tokenizedGroupDataframe.pkl", "wb") as f:
#    pickle.dump(tokenizedGroupData, f)

801


Epoch 1/3
2324/2324 [==============================] - 32s 12ms/step - loss: 0.5951 - mse: 385.9896 - mae: 14.3964 - mape: 271627808.0000 - accuracy: 0.0023
Epoch 2/3
2324/2324 [==============================] - 29s 12ms/step - loss: 0.2223 - mse: 134.8881 - mae: 8.0203 - mape: 118693400.0000 - accuracy: 0.0052
Epoch 3/3
2324/2324 [==============================] - 28s 12ms/step - loss: 0.1833 - mse: 109.6337 - mae: 7.1772 - mape: 84391008.0000 - accuracy: 0.0061


/home/jaredrussell/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


/home/jaredrussell/gitRepos/SCA-ML-grade/Model-Generation/SCA-Tokenizer/
/home/jaredrussell/gitRepos/SCA-ML-grade/Model-Generation/SCA-Tokenizer/.gitignore
/home/jaredrussell/gitRepos/SCA-ML-grade/Model-Generation/SCA-Tokenizer/TokenizerManager.py
/home/jaredrussell/gitRepos/SCA-ML-grade/Model-Generation/SCA-Tokenizer/.DS_Store
/home/jaredrussell/gitRepos/SCA-ML-grade/Model-Generation/SCA-Tokenizer/main.py
/home/jaredrussell/gitRepos/SCA-ML-grade/Model-Generation/SCA-Tokenizer/.git
/home/jaredrussell/gitRepos/SCA-ML-grade/Model-Generation/SCA-Tokenizer/CaMlSupportingClasses.py
/home/jaredrussell/gitRepos/SCA-ML-grade/Model-Generation/SCA-Tokenizer/__pycache__/
/home/jaredrussell/gitRepos/SCA-ML-grade/Model-Generation/SCA-Tokenizer/__pycache__/CLPTokenizer.cpython-310.pyc
/home/jaredrussell/gitRepos/SCA-ML-grade/Model-Generation/SCA-Tokenizer/__pycache__/TokenizerManager.cpython-310.pyc
/home/jaredrussell/gitRepos/SCA-ML-grade/Model-Generation/SCA-Tokenizer/__pycache__/CaMlSupportingCla

tar: Removing leading `/' from member names
tar: Removing leading `/home/jaredrussell/gitRepos/SCA-ML-grade/Model-Generation/../' from member names
